In [1]:
import time, facebook, sys
import pandas as pd
sys.path.insert(1, '../../../scripts/')
from s3_support import *

In [2]:
APP_ID = "380047073336642"
APP_SECRET = "667d0e2781b50e38e38a8cfd1a8284b9"

In [18]:
forms = redshift_query_read("select id, org, type, path from form where status=1", schema="production")
facebook_shares = redshift_query_read("select * from social_shares_facebook", schema="public")

In [6]:
seconds_delay = 200. / 60.
counter = 0
print("FACEBOOK SPIDER: crawling {} forms with a {} second delay between requests".format(len(forms), seconds_delay))

FACEBOOK SPIDER: crawling 23655 forms with a 3.3333333333333335 second delay between requests


# crawling URL's

In [19]:
api = facebook.GraphAPI(access_token="{}|{}".format(APP_ID, APP_SECRET), version="2.12")
test_counter = 0

for _, form in forms.iterrows():
    # build form URL to search for
    if form['type'] == 3:
        # p2p2 formatting
        form_url = "https://secure.qgiv.com/event/{}".format(form['path'])
    else:
        # qgiv formatting
        form_url = "https://secure.qgiv.com/for/{}".format(form['path'])

    if form_url is not None and form_url != "":
        fb_query = "v9.0?id={}&fields=engagement".format(form_url)
        
        try:
            # query facebook for search results
            results = api.request(fb_query)

            # process & store results
            if 'engagement' in results:
                # process url info
                data = {
                    'form': form['id'],
                    'url': form_url,
                    'reaction_count': results['engagement']['reaction_count'],
                    'comment_count': results['engagement']['comment_count'],
                    'share_count': results['engagement']['share_count']
                }

                # store url info & keep most recent
                facebook_shares = facebook_shares.append(pd.DataFrame([data]), sort=False)
                test_counter += 1
        except Exception as e:
            print("Exception: {}".format(e))

        # delay until next iteration
        time.sleep(seconds_delay)
        if test_counter > 100:
            break

In [20]:
facebook_shares

,form,url,reaction_count,comment_count,share_count
0,1,https://secure.qgiv.com/for/cfsun,181,9,1201
0,18,https://secure.qgiv.com/for/cipherint,0,0,0
0,50,https://secure.qgiv.com/for/hsdc,0,0,0
0,66,https://secure.qgiv.com/for/coak,0,0,0
0,114,https://secure.qgiv.com/for/ucptampa,0,0,1
0,226,https://secure.qgiv.com/for/altamedhealthservi...,0,0,0
0,338,https://secure.qgiv.com/for/polktheatre,0,0,0
0,354,https://secure.qgiv.com/for/muslimcs,0,0,0
0,386,https://secure.qgiv.com/for/MRLAJF6H6STQ5GEMH2ZP,0,0,0
0,434,https://secure.qgiv.com/for/tsotcevents,0,0,0


# org pages

In [27]:
social_settings = get_dataframe_from_file("qgiv-stats-data", "social_settings.exp.csv")
social_settings = social_settings[~social_settings['facebookURL'].isna()]

In [28]:
cols = ['id', 'form', 'facebookURL', 'enableFBFundraiser', 'twitterURL']
social_settings[cols].tail()

,id,form,facebookURL,enableFBFundraiser,twitterURL
11118,86246,957344,https://www.facebook.com/MWLimits/,0,https://twitter.com/mwlimits
11126,86254,957347,http://facebook.com/raleighmasjid,0,http://twitter.com/raleighmasjid
11138,86266,957424,https://www.facebook.com/dsawisconsin,0,https://twitter.com/DSAWisconsin
11151,86279,957452,https://www.facebook.com/BrotherWolfAnimalRescue,0,https://twitter.com/BrotherWolfAR
11173,86301,957485,https://www.facebook.com/JuniorAchievementofNe...,1,https://twitter.com/janewyork?lang=en


In [31]:
api = facebook.GraphAPI(access_token="{}|{}".format(APP_ID, APP_SECRET), version="2.12")
fb_query = "pages/search?q={}&fields=id,name,engagement".format("MWLimits")
results = api.request(fb_query)

GraphAPIError: (#10) This endpoint requires the 'pages_read_engagement' permission or the 'Page Public Content Access' feature or the 'Page Public Metadata Access' feature. Refer to https://developers.facebook.com/docs/apps/review/login-permissions#manage-pages, https://developers.facebook.com/docs/apps/review/feature#reference-PAGES_ACCESS and https://developers.facebook.com/docs/apps/review/feature#page-public-metadata-access for details.